# Multi-head Latent Attention

下面是一张**MLA（Multi-Latent Attention）从输入到输出的文字逻辑图（附维度 shape）**，清晰地看到整个流程中各张量的形状变化与计算路径：

---

## 🧠 MLA Attention 逻辑流（以 DeepSeek-V2 参数为例）

```
──────────────────────────────────────────────
【输入阶段】
──────────────────────────────────────────────
输入隐藏状态（每个 Token）
    h_t ∈ ℝ^{5120}      ← Transformer 隐层输出
──────────────────────────────────────────────
↓ 下投影（Dim Reduction）
──────────────────────────────────────────────
通过线性投影矩阵 W^{D_Q} ∈ ℝ^{5120×1536}
    c_t^Q = W^{D_Q} · h_t
输出：
    c_t^Q ∈ ℝ^{1536}      ← 压缩后的低秩 latent 表示
──────────────────────────────────────────────
↓ 多头展开（Multi-head Reconstruction）
──────────────────────────────────────────────
通过上投影矩阵 W^{U_Q} ∈ ℝ^{1536×(n_h·d_h)} = ℝ^{1536×16384}
    q_t^c = W^{U_Q} · c_t^Q
输出：
    q_t^c ∈ ℝ^{16384}     ← 多头拼接后的高维 Q 向量
──────────────────────────────────────────────
↓ 多头拆分（Head Split）
──────────────────────────────────────────────
将 q_t^c reshape → [n_h, d_h] = [128, 128]
得到每个注意力头的 Q 向量：
    Q_head[i] ∈ ℝ^{128} ,  i = 1...128
──────────────────────────────────────────────
↓ 与 K,V 做注意力计算
──────────────────────────────────────────────
Attention_i = softmax( (Q_i K_i^T)/√d_h ) · V_i
每个 head 独立计算 → 拼接所有结果：
    Concat(Attention_1, ..., Attention_128) ∈ ℝ^{16384}
──────────────────────────────────────────────
↓ 输出投影（Output Projection）
──────────────────────────────────────────────
通过 W^O ∈ ℝ^{16384×5120}
恢复到原始隐层维度：
    h'_t = W^O · Concat(Attention_i)
输出：
    h'_t ∈ ℝ^{5120}
──────────────────────────────────────────────
【输出阶段】
──────────────────────────────────────────────
返回到 Transformer 主干
──────────────────────────────────────────────
```

---

## ✨ 总结层次结构

| 阶段  | 名称                    | 输入 shape  | 输出 shape  | 操作           |
| --- | --------------------- | --------- | --------- | ------------ |
| 1️⃣ | 下投影 (Down Projection) | 5120      | 1536      | 压缩为低秩 latent |
| 2️⃣ | 上投影 (Up Projection)   | 1536      | 16384     | 恢复多头 Q 拼接向量  |
| 3️⃣ | 多头拆分 (Head Split)     | 16384     | [128×128] | 分配给每个 head   |
| 4️⃣ | Attention 计算          | [128×128] | [128×128] | 计算多头注意力      |
| 5️⃣ | 拼接与输出投影               | [128×128] | 5120      | 合并输出到原维度     |

---

## 💡 直观理解

* **5120 → 1536**：低秩压缩（减少显存）
* **1536 → 16384 → [128×128]**：多头展开，恢复多视角注意力
* **注意力后再降回 5120**：保持与主干维度一致

---

如果你希望我在这张文字逻辑图的基础上，加上 **Key / Value 路径（K,V 的下投影和上投影流程）**，我可以补充完整的 **MLA Attention 全路径结构图（Q,K,V三支路）**，是否需要？
